# Загрузка Pandas и очистка данных

In [58]:
import pandas as pd
import ast

In [59]:
df = pd.read_csv('main_task.csv')

## Анализ источника данных

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


## Предобработка

#### Вспомогательные функции

In [61]:
col_cuisines = 'Cuisine Style'
col_rest_id = 'Restaurant_id'
col_price = 'Price Range'
col_rating = 'Rating'
col_reviews = 'Number of Reviews'


def fill_cuisines(cuisines):
    """Преобразует строку, содержащую список в объект списка"""

    return ['Unknown'] if pd.isna(cuisines) else ast.literal_eval(cuisines)

In [62]:
df[col_cuisines] = df.apply(lambda row: fill_cuisines(row[col_cuisines]), axis=1)

## Очистка данных

#### Заполняем пропуски

In [63]:
df[col_reviews] = df['Number of Reviews'].fillna(df[col_reviews].mean())


## Feature Engineering


#### Создаем dummy признаки для ценовой категории

In [64]:
price_ranges = pd.get_dummies(df[col_price])
price_ranges.columns = ['Inexpensive', 'Middle Price', 'Expensive']

for c in price_ranges.columns:
    df[c] = price_ranges[c].values


#### Создаем dummy признак количества кухонь

In [65]:
df[col_cuisines] = df.apply(lambda row: len(row[col_cuisines]), axis=1)

#### Удаляем столбцы с dtype Object

In [66]:
object_columns = []

for c in df.columns:
    if(df[c].dtype == 'object' and c != col_rest_id):
        object_columns.append(c)

df.drop(object_columns, axis=1, inplace=True)


# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [67]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop([col_rest_id, col_rating], axis = 1)
y = df[col_rating]

In [68]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [69]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [70]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [71]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [72]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.4097357992063492
